/// Définition du Crawler de REUTERS ///

In [18]:
# Import des packages nécessaires au crawling d'un site html
from bs4 import BeautifulSoup
import requests

In [19]:
# Definition du dictionnaire companies:prefix, des companies à crawler
companies_dic = dict()
companies_dic["Airbus"] = "AIR.PA"
companies_dic["Danone"] = "DANO.PA"
companies_dic["LVMH"] = "LVMH.MI"

// Définitions des fonctions "unitaires" //

In [20]:
# Definition de la fonction de crawling de la valeur de l'action
def get_stockvalue(soup):
    stock_value = float(
        soup.find_all("div", {"class": "sectionQuoteDetail"})[0]
            .find_all("span")[1]
            .text.replace("\t", "").replace("\n", ""))

    return stock_value

In [21]:
# Definition de la fonction de crawling du taux de change de l'action
def get_stockchange(soup):
    stock_change = float(
        soup.find_all("div", {"class": "sectionQuoteDetail"})[1]
            .find("span", {"class": "valueContentPercent"})
            .text
            .replace("\n", "").replace("\t", "").replace("(", "").replace(")", "").replace("%", ""))

    return stock_change

In [22]:
# Definition de la fonction de crawling des ventes au Q4 2018
def get_salesq42018(soup):
    sales_q4 = float(
        soup.find_all("table")[0]
            .find_all('tr', {"class": "stripe"})[0]
            .find_all('td', {'class': 'data'})[1]
            .text.replace(',', ''))

    return sales_q4

In [23]:
# Definition de la fonction de crawling du % de Shares owned
def get_sharesowned(soup):
    shares_owned = float(
        soup.find("td", text="% Shares Owned:")
            .parent()[2]
            .text.replace("%", ""))

    return shares_owned

In [24]:
# Definition de la fonction de crawling des dividend yield, pour l'entreprise, l'industrie et le secteur
def get_dividendyield(soup):
    dividend_yield = list()
    dividend_yield.append(
        float(
            soup.find("td", text="Dividend Yield")
                .parent()[1].text))

    dividend_yield.append(
        float(
            soup.find("td", text="Dividend Yield")
                .parent()[2].text))

    dividend_yield.append(
        float(
            soup.find("td", text="Dividend Yield")
                .parent()[3].text))

    return dividend_yield

In [32]:
# Définition de la fonction principale qui lance le crawler pour la liste des entreprises demandées en argument
def crawler(companies):
    #Si une seule company est passée en argument : un crawler est lancé
    if type(companies) == str:
        print("***Informations financières de " + companies + "***\n")
        crawl(companies)
    
    #Si plusieurs companies à crawler, lancement d'un crawler pour chaque company
    elif type(companies) == list:
        for company in companies:
            print("***Informations financières de " + company + "***\n")
            crawl(company)
            print("\n\n")
    
    # Si l'argument du crawler n'est ni une company, ni une liste de companies, on demande de resaisir l'input
    else:
        print("Veuillez renseigner le nom d'une entreprise ou une liste d'entreprises")

In [33]:
# Définition de la fonction de crawl d'un site
def crawl(company):
    # ici on lit le prefix financier associé à l'entreprise pour trouver le site sur REUTERS.COM
    company_prefix = companies_dic[company] 
    
    # Definition de l'URL et de l'objet site
    url = 'https://www.reuters.com/finance/stocks/financial-highlights/' + company_prefix
    site = requests.get(url).text
    
    # Construction de l'objet soup
    soup = BeautifulSoup(site, 'html.parser')

    # Récupération des outputs souhaités, chacun obtenu par une fonction spécifique
    stock_value = get_stockvalue(soup) 
    stock_change = get_stockchange(soup) 
    sales_q4 = get_salesq42018(soup)
    dividend_yield = get_dividendyield(soup)
    shares_owned = get_sharesowned(soup)
    
    # Affichage des résultats du crawl
    print(
        "Ventes - Q4 2018 : " + str(sales_q4) + "M€" + "\n" +
        "Cours de l'action : " + str(stock_value) + "€" + "\n" +
        "Taux de change de l'action : " + str(stock_change) + "%" + "\n" +
        "% Shares Owned : " + str(shares_owned) + "%" + "\n" +
        "Dividend yield de " + str(company) + " : " + str(dividend_yield[0]) + "\n" +
        "Dividend yield de l'industrie : " + str(dividend_yield[1]) + "\n" +
        "Dividend yield du secteur : " + str(dividend_yield[2]))

In [34]:
crawler("Airbus")

***Informations financières de Airbus***

Ventes - Q4 2018 : 23493.0M€
Cours de l'action : 98.99€
Taux de change de l'action : -3.42%
% Shares Owned : 43.53%
Dividend yield de Airbus : 1.45
Dividend yield de l'industrie : 1.34
Dividend yield du secteur : 1.64
